## Extract the Data

In [ ]:
import pyarrow.parquet as pq

trips = pq.read_table('data/yellow_tripdata_2014-01.parquet')
df = trips.to_pandas()

## Take a Glimpse of the Data and Clear

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index

In [ ]:
df.head(10)

In [ ]:
df.fillna({'airport_fee': 0}, inplace=True)

In [ ]:
df.dropna(subset=['store_and_fwd_flag'], inplace=True)

In [ ]:
df[['store_and_fwd_flag']]

## Transform the Data

### Datetime

In [ ]:
datetime_dim = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].reset_index(drop=True)
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim["tpep_pickup_datetime"].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim["tpep_pickup_datetime"].dt.year
datetime_dim['pick_weekday'] = datetime_dim["tpep_pickup_datetime"].dt.weekday

datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim["tpep_dropoff_datetime"].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim["tpep_dropoff_datetime"].dt.year
datetime_dim['drop_weekday'] = datetime_dim["tpep_dropoff_datetime"].dt.weekday

In [ ]:
datetime_dim.head()

In [ ]:
datetime_dim['datetime_id'] = datetime_dim.index

In [ ]:
datetime_dim.head()

In [ ]:
datetime_dim = datetime_dim[
    ['datetime_id',
     'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
     'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]

In [ ]:
datetime_dim.head()

In [ ]:
datetime_dim.info()

### Passenger Count

In [ ]:
passenger_count_dim = df[['passenger_count']].reset_index(drop=True)

In [ ]:
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index

In [ ]:
passenger_count_dim = passenger_count_dim[['passenger_count_id', 'passenger_count']]

In [ ]:
passenger_count_dim.head()

### Trip Distance

In [ ]:
trip_distance_dim = df[['trip_distance']].reset_index(drop=True)

In [ ]:
trip_distance_dim.head()

In [ ]:
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index

In [ ]:
trip_distance_dim = trip_distance_dim[['trip_distance_id', 'trip_distance']]
trip_distance_dim.head()

### Rate Code

In [ ]:
rate_code_dim = df[['RatecodeID']].reset_index(drop=True)

In [ ]:
rate_code_dim.head()

In [ ]:
rate_code_dim['rate_code_id'] = rate_code_dim.index

In [ ]:
rate_code_name = {
    1: "Standard Rate",
    2: "JFK",
    3: "Newark",
    4: "Nassau or Westchester",
    5: "Negotiated fare",
    6: "Group ride"
}

rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_name)
rate_code_dim = rate_code_dim[['rate_code_id', 'RatecodeID', 'rate_code_name']]

### Payment Type

In [ ]:
payment_type_dim = df[['payment_type']].reset_index(drop=True)

In [ ]:
payment_type_dim['payment_type_id'] = payment_type_dim.index

In [ ]:
payment_type_dim['payment_type_id'].value_counts()

In [ ]:
payment_type_name = {
    1: "Credit card",
    2: "Cash",
    3: "No Charge",
    4: "Dispute",
    5: "Unknown",
    6: "Voided trip"
}
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)

In [ ]:
payment_type_dim = payment_type_dim[['payment_type_id', 'payment_type_name', 'payment_type']]
payment_type_dim.head()

### Merge into a Fact Table

In [ ]:
fact_table = (df.head(10).
              merge(datetime_dim.head(10), left_on='trip_id', right_on='datetime_id').
              merge(passenger_count_dim.head(10), left_on='trip_id', right_on='passenger_count_id').
              merge(trip_distance_dim.head(10), left_on='trip_id', right_on='trip_distance_id').
              merge(payment_type_dim.head(10), left_on='trip_id', right_on='payment_type_id').
              merge(rate_code_dim.head(10), left_on='trip_id', right_on='rate_code_id'))[[
    'trip_id', 'VendorID', 'datetime_id', 'passenger_count_id', 'trip_distance_id', 
    'rate_code_id', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
    'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
    'improvement_surcharge', 'total_amount'
]]

In [ ]:
fact_table.head()

In [ ]:
fact_table.dropna(inplace=True)

In [ ]:
fact_table[['store_and_fwd_flag']].value_counts()